In [ ]:
import pandas as pd
import pysolr


In [ ]:
# url = "http://b2find.eudat.eu/solr/select"
url = "http://localhost:8983/solr/ckan"


In [ ]:
facet = "extras_PublicationYear"

solr_params = {
    'echoParams': 'none',
    'rows': 1,
    'wt': 'json',
    'q': '*',
    'fq': f"{facet}:[2000 TO 2010]",
    'facet': 'true',
    'facet.sort': 'index',
    'facet.limit': -1,
    'facet.field': [facet],
}

In [ ]:
solr = pysolr.Solr(url)

In [ ]:
results = solr.search(**solr_params)


In [ ]:
# results.raw_response

In [ ]:
#results.facets

In [ ]:
result_counts = results.facets["facet_fields"][facet]
x = result_counts[0::2]
y = result_counts[1::2]

In [ ]:
# skip zero counts at the start and end
start = list(map(lambda val: val > 0, y)).index(True)
end = list(map(lambda val: val > 0, reversed(y))).index(True)
end = len(y) - end

x = x[start:end]
y = y[start:end]

In [ ]:
df = pd.DataFrame(list(zip(x, y)), columns =['year', 'counts'])
df["year"]=pd.to_numeric(df["year"], downcast="signed")
df

In [ ]:
from bokeh.io import show
from bokeh.models import CustomJS, RangeSlider


callback = CustomJS(code="""
    console.log('date_range_slider: value=' + this.value, this.toString())
""")
start=df.year[0]
end=df.year[len(df)-1]
if start == end:
    end = start + 1
date_range_slider = RangeSlider(
        title="Publication Year",
        value=(start, end),
        start=start, 
        end=end,
)

date_range_slider.js_on_change("value_throttled", callback)

show(date_range_slider)